Copyright Scott Jensen, San Jose State University

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">This ds4all notebook</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Scott Jensen,Ph.D.</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

# Basic Wrangling

In this notebook we are going to introduce some basic ideas about PySpark (since we will be using Python).  
The documentation for the classes and functions in that module can be found <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html" target="_blank">here</a>.
If you click on that link, it can be a bit intimidating - remember that page is documenting *every* method and function in PySpark.  At the top of that
page are the major classes in the PySpark SQL module.  We won't be using most of the methods and functions, but in this exercise and future notebooks and examples we will walk through some you will be using in your project.

We will start with the SparkSession, which is created by Databricks when you spun up your cluster and it's always named `spark` by convention (not just in Databricks - that's a Spark convention).

## Start With A Question: What metro areas are in the yelp dataset?  

The Yelp Dataset Challenge tells us there is data for 8 metro areas in the dataset, but it does not tell us 
which metro areas.  This means we need to do some data wrangling to find out.  We need to ask the data 
which metro areas are included in the data.  If you have read the *Principles of Data Wrangling*, is this set based or individual value based profiling?

First, we need to read our data. From the metadata provided by Yelp, we know the business data includes 
information on where each business is located: latitude and longitude, postal code, state, city, neighborhood, address so we
will start by reading the Yelp business data into a DataFrame.  

We will walk through reading our JSON files (zipped up in the bzip2 format). 
In future exercises we will just say we are reading the data - from experience you will know how to do that (or know which
of your notebooks to look at for an example).

To read the data file, we start with the `read` method from our `SparkSession` variable named `spark`.

If you look at the PySpark documentation for the **<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.read.html#pyspark-sql-sparksession-read" target="_blank">read</a>** method, you will see that it returns a `DataFrameReader` 
which "can be used to read data in as a DataFrame".  If we entered the following in a cell, the variable we are creating on the left side of the = sign (named `dfr`) would be populated with an instance of a `DataFrameReader`.

`dfr = spark.read`

However, we want to read the business data into a DataFrame, not create a DataFrameReader, so we won't stop there.  Let's take a look at the documentation for 
a **<a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#input-and-output" target="_blank">DataFrameReader</a>**, click on it.  The first method you will probably see for 
a `DataFrameReader` is the `csv()` method which reads data in from a csv file such as our data from the 
Social Security Administration.  However, we have a JSON file, 
so scroll on down to the **<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameReader.json.html?highlight=dataframereader#pyspark-sql-dataframereader-json" target="_blank">json</a>** method.
You will see there are a lot of possible options, but for our purposes the defaults are fine, we just need to provide a path.  
In the cell below, add the path to your business data as a string (meaning the path is enclosed in quotes).  If you are not sure what that
path is, go take a look at your DBFS and come back here.  To see what is in DBFS on your account, click on the `Data` icon in the toolbar on the left in Databricks, and then click the `Create Table` button.
You may be saying, "but I'm not creating a table at the moment", however this is the path to seeing what you have stored in DBFS. In the `Create New Table` screen that appears, click on the `dBFS` button in the row of buttons on the top.  You are now in your DBFS, and if you loaded the Yelp data they way we discussed in class, it's in the `/yelp` directory, so click on that.  whenever you have a file or directory selected in DBFS, the path to get there is listed at the bottom of the screen, so you can copy and paste that path into the code below.

The result of the `json` method of the `DataFrameReader` is a DataFrame, so we assign the result to a variable named `df_business`.
Keep in mind what is happening here.  The code `spark.read` generates a `DataFrameReader`, so the "dot" after that code is saying 
to call the `json` method from that class on that `DataFrameReader` which was created.

## Step 1: Loading the data

The first line reads your Yelp business data file.  Be sure it has the correct path to your data.

The last two lines of the cell print out a count of the number of businesses in the data file and the schema inferred by Spark for the data file.

In [0]:
df_business = spark.read.json('/yelp/business.bz2')
print ("record count:", df_business.count() )
df_business.printSchema()

md ## Step 2: What does our data look like?

The DataFrame class has a method named `show()` that will show us a number of rows / records in our data; 20 rows is the default.  Since some fields, such as the business address or the text of a review can be long, the default is to truncate how many characters is shown for each column and only show the first 20 characters (truncate is another way of saying "chop off").  We can include optional parameters to say how many rows we want and whether to truncate the fields (or how many characters to truncate at).

In the following cell we are going to show the following fields from the Yelp business data:
* business_id
* name
* city
* state
* review_count
* stars
* hours (when they are open)
* RestaurantsPriceRange2 (within attributes, this is a value 1 - 4 to indicate how expensive it is)

The DataFrame method we will use is named `select` which is documented **<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.select.html#pyspark-sql-dataframe-select" target="_blank">here</a>**.

The `select` method takes the names of fields in a DataFrame as its parameters (as many as you want back), and returns a DataFrame with those columns.  Since we want to just
see the data, we will call the `show` method directly on the DataFrame generated by `select`, we just need to include the dot in between to say "pass our new DataFrame to the `show` method".

In [0]:
df_business.select("business_id", "name", "city", "state", "review_count", "stars", "hours", "attributes.RestaurantsPriceRange2").show()

## Step 3: Profiling for the metro areas

The closest field in the data to the concept of a metro area is a city, so let's see how many cities we have in the data.

For the above cell, name the resulting DataFrame `df_busSelected`

Using our new DataFrame, we will select the `city` and `state` fields.

We really only want each city and state combination listed once, so the Flying Elephants at PDX is in Portland Oregon (OR), but we only want that city listed once, not once for each business.  If there is a Portland Georgia, we would also want that listed.
To accomplish this, there is a DataFrame method named `distinct` which returns only the distinct rows - see the documentation **<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.distinct.html#pyspark-sql-dataframe-distinct" target="_blank">here</a>**.

Because we are humans (assuming ...), we like data to be sorted, so we will add the DataFrame **<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.orderBy.html#pyspark-sql-dataframe-orderby" target="_blank">orderBy</a>** method and sort first by state and then by city.

In [0]:
df_busSelected.select("state", "city").\
distinct().orderBy("state", "city").show(100, truncate=False)

## There are more than 8 cities

Our profiling provided valuable information - cities are not each considered a metro area.

We will take a step back - how many businesses are there in each state?  Is there one metro area per state?  Are there multiple metro areas in a state (and also in our data)?  For example, if our data included
California, Los Angeles, San Francisco, and San Diego would all be separate metro areas in the data.

First let's look at the states.

The DataFrame class has a method named `groupBy` that will group our data based on a field or fields we specify 
(documentation is <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.groupBy.html#pyspark-sql-dataframe-groupby" target="_blank">here</a>).

You can think of `groupBy` as being like if you had a deck of cards and you wanted to group them by suit (hearts, clubs, etc.).  We want to 
group the business data by state.  This may seem similar to using `distinct`, but where that method got rid of duplicates, `groupBy` allows us to 
do calculations on each group.  

The difference can be explained with a deck of cards.  If you asked your younger sister, "what are the distinct suits in a deck of cards", 
you would get four values back: diamonds, spades, clubs, and hearts.  If you asked her to group the cards by suit, you could then ask her to count how many there are in 
each suit.  If all of the cards are there, the answer would be 13 for each suit.

After using the DataFrame `groupBy` method, we no longer have a DataFrame.  As stated in the documentation, 
we now have an instance of the `GroupedData` class as documented <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.html#pyspark-sql-groupeddata" target="_blank">here</a>.

The `GroupedData` class has a number of methods we can use to calculate information about each group (e.g., the mean, average, etc.). We want to get a count of the number of 
businesses in each state, so we will use the `count` method documented <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.count.html#pyspark-sql-groupeddata-count" target="_blank">here</a>,
which returns a DataFrame, so we can call the `show` method on it.

## Step 4: Counting businesses by state
Add the code to sort with the state containing the most businesses listed first.

In [0]:
df_busSelected.select("state").groupBy("state").count().show(100, truncate=False)

## Step 5: What Metro Area(s) Are Included from Ohio?
When we showed the data in the business DataFrame earlier, there was a business named "Boxwood Biscuit" in the state of Ohio (state code OH), but how many businesses are there in the data from Ohio? Get a count of the businesses in each *city* in that state and sort the cities in descending order.  This could tell us if there is a specific city that defines the metro area in the data from Ohio.

To do this, you first need one other DataFrame method, **<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.filter.html#pyspark-sql-dataframe-filter" target="_blank">filter</a>**.  Whereas `select` allowed us to choose to include only specific columns, `filter` allows us to include only specific rows that match some
criteria specified in our filter.  The following cell filters our business data to include only the rows for those businesses in Ohio.  If you already know SQL, this should sound very similar to the WHERE clause in a SQL query, and in fact you can use `where` as a substitute for `filter` in PySpark

### Your First Mission:
In the following cell we are filtering to get only the rows where the state is Ohio (OH).

We want to count the number of businesses in each *city* in Ohio and show the count so the city with the most businesses is listed first.

In [0]:
# Edit the code shown here
df_busSelected.filter(df_busSelected.state == 'OH').show()


# Your Mission: Find the 8 Metro Areas

Find the top city for each of the top 9 states (it drops off fast after that).

Use markdown to record the state / metro area combinations (**complete the table** - Ohio is listed, but be sure it is in the correct order in the final markdown table):

| State | Metro Area | Size |
| ------- | ------------ |
| OH  |  Columbus  |  11258  | 



The cells below should include your code for determining the metro areas for each of the top 9 states.  


## Extra Credit Option
You can copy and modify the code for Ohio, or use a loop to determine the 
numbers and print them for the top 9 states.  To be a loop, you cannot just have the same code nine times (that's not a loop).  If you have heard of Nascar stock car racing, it's a popular autoracing 
series where cars go in a loop that's roughly 2 miles long hundreds of times (you may find that fascinating or think it just sounds like dad circling looking for a parking spot at the mall). In the Nascar case, the loop is 2 miles long, but the cars go hundreds of miles.  In your code, the loop is through 9 states, but the loop only has code that prints a single state.

If you are familiar with SQL, you may be thinking it could be done more easily in SQL, and you would be correct, but we will cover SQL later in other notebooks.

To earn the extra-credit:
* You **must** use markdown to describe what your code is doing (no markdown == no credit)
* It must be **your** code
* It **must** be a loop
* It must be uploaded by the due date

A couple hints:

* **<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.limit.html#pyspark-sql-dataframe-limit" target="_blank">limit</a>** will limit the number of rows returned in a DataFrame to whatever value X is set to
* **<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.collect.html#pyspark-sql-dataframe-collect" target="_blank">collect</a>** will return a python list of the rows in the DataFrame, where each list item is a row

In [0]:
# Add your code here 
# Be sure to also update the markdown cell above, including the first 9 states (not just Ohio)


# Deliverable

* After completing your mission for the top 9 states, publish your notebook and upload the URL as the deliverable for this assignment.
* It is recommended that you **test** the URL you submit.  To do this, first log out of Databricks and then put your URL in the browser.  Did you see what you expected? (If you are still logged into Databricks, the wrong URL can also work.)

#### We will come back to the question of identifying metro areas in the data later in the semester when we are working in Tableau.